In [6]:
#import tensorflow as tf 500 и 200 нейронов
#from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_openml

In [7]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [8]:
class MultiLayerPerceptron:
  def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01, epochs=1000):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.input_size = input_size
    self.hidden_sizes = hidden_sizes # Список с количеством сумматоров в каждом скрытом слое
    self.output_size = output_size

    # Инициализация весов и смещений
    self.weights = []
    self.biases = []
    
    # Начальное количество входов
    layer_input_size = input_size
    
    for hidden_size in hidden_sizes:
      self.weights.append(np.random.uniform(size=(layer_input_size, hidden_size), low=-0.5, high=0.5))
      self.biases.append(np.random.uniform(low=-0.5, high=0.5, size=(hidden_size,)))
      layer_input_size = hidden_size
    
    self.weights.append(np.random.uniform(size=(layer_input_size, output_size), low=-0.5, high=0.5))
    self.biases.append(np.random.uniform(low=-0.5, high=0.5, size=(output_size,)))

  def activation_function(self, net_input):
    return 1 / (1 + np.exp(-net_input))

  def fit(self, features, targets):
    n_examples = features.shape[0]
    
    for _ in range(self.epochs):
      for example_index, example_features in enumerate(features):
        # Прямое распространение
        activations = [example_features]
        for weight, bias in zip(self.weights, self.biases):
          net_input = np.dot(activations[-1], weight) + bias
          activation = self.activation_function(net_input)
          activations.append(activation)

        # Обработка ошибок (обратное распространение)
        error = targets[example_index] - activations[-1]
        for i in reversed(range(len(self.weights))):
          output = activations[i + 1]
          delta = error * output * (1 - output) # Производная сигмоиды
          if i > 0: # Если это не последний слой, мы должны учитывать предыдущее состояние
            error = np.dot(delta, self.weights[i].T)
          self.weights[i] += self.learning_rate * np.outer(activations[i], delta)
          self.biases[i] += self.learning_rate * delta

  def predict(self, features):
    for weight, bias in zip(self.weights, self.biases):
      features = self.activation_function(np.dot(features, weight) + bias)
    return features


In [23]:
mnist = fetch_openml('mnist_784')
X, y = mnist['data'], mnist['target'].astype(int)

In [24]:
X = X.astype(float)

In [28]:
# Предобработка данных
X /= 255.0 # Нормализация

# Одна горячая кодировка меток
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
   
X_train = X_train.values
y_train = y_train.toarray()
   

# Моделирование
mlp = MultiLayerPerceptron(input_size=784, hidden_sizes=[128, 64], output_size=10)
mlp.fit(X_train, y_train)

# Прогноз
y_pred_train = mlp.predict(X_train)
y_pred_test = mlp.predict(X_test)

# Оценка точности
train_accuracy = accuracy_score(np.argmax(y_train, axis=1), np.argmax(y_pred_train, axis=1))
test_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_test, axis=1))

print(f"Train accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")

KeyboardInterrupt: 

In [ ]:
# Обучение модели SVC
svc_pipeline = make_pipeline(StandardScaler(), SVC(C=1.0, gamma='scale'))
svc_pipeline.fit(X_train, y_train.argmax(axis=1))

# Прогноз
svc_predictions = svc_pipeline.predict(X_test)

# Оценка точности
svc_accuracy = accuracy_score(y_test.argmax(axis=1), svc_predictions)
print(f"SVC test accuracy: {svc_accuracy:.2f}")